In [1]:
from google.colab import drive
drive.mount('/content/drive')
print('mount success')

Mounted at /content/drive
mount success


###Installation

In [98]:
!pip install keras_preprocessing

In [99]:
!pip install dill
!pip show h5py

Name: h5py
Version: 3.9.0
Summary: Read and write HDF5 files from Python
Home-page: 
Author: 
Author-email: Andrew Collette <andrew.collette@gmail.com>
License: BSD-3-Clause
Location: /usr/local/lib/python3.10/dist-packages
Requires: numpy
Required-by: h5netcdf, tensorflow


In [100]:
# Imports for JupyterLite
%pip install -q ipywidgets

### Imports

In [101]:
import os
import json
import functools
import operator
import time
import h5py
import numpy as np
from keras.layers import Input, LSTM, Dense
from keras.models import Model, load_model
import joblib

In [102]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [103]:
def f(x):
  return x

###Data and Models

In [104]:
# test path
test_path = '/content/drive/Shareddrives/msvd-dataset/test'

# test features
test_features_path = '/content/drive/Shareddrives/msvd-test-feats/test/custom_feat'

In [105]:
test_features_path = '/content/drive/Shareddrives/msvd-test-feats/TEST_FEAT/custom_feat'
test_videos_path = '/content/drive/Shareddrives/msvd-test-feats/TEST_VIDEOS'
test_videos_filepath = '/content/drive/Shareddrives/msvd-test-feats/test_videos.txt'
models_path = '/content/drive/Shareddrives/FYP-models'

In [106]:
models_list = os.listdir(models_path)
models_list

['without_cust_vocab_att_3700',
 'without_cust_vocab_1500',
 'without_cust_vocab_3500',
 'without_cust_vocab_3700',
 'without_cust_vocab_att_1500']

In [107]:
video_ids = os.listdir(test_videos_path)
video_ids

['fJr2evLANsE_0_10.avi',
 'gqxpGOHUH9k_113_119.avi',
 'fr9H1WLcF1A_141_148.avi',
 'g36ho6UrBz0_5_20.avi',
 'hJFBXHtxKIc_204_209.avi']

## MODEL FOR INFERENCE

In [108]:
def inference_model():
    """Returns the model that will be used for inference"""
    with open(os.path.join(save_model_path, 'tokenizer' + str(num_decoder_tokens)), 'rb') as file:
        tokenizer = joblib.load(file)
    # loading encoder model. This remains the same
    inf_encoder_model = load_model(os.path.join(save_model_path, encoder_name))

    # Manually compile the encoder model
    inf_encoder_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


    # inference decoder model loading
    decoder_inputs = Input(shape=(None, num_decoder_tokens))
    decoder_dense = Dense(num_decoder_tokens, activation='softmax')
    decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
    decoder_state_input_h = Input(shape=(latent_dim,))
    decoder_state_input_c = Input(shape=(latent_dim,))
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    inf_decoder_model = Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)
    inf_decoder_model.load_weights(os.path.join(save_model_path, decoder_name))
    return tokenizer, inf_encoder_model, inf_decoder_model


# TESTING


In [109]:
class VideoDescriptionInference(object):
    """
        Initialize the parameters for the model
        """
    def __init__(self):
        self.latent_dim = latent_dim
        self.num_encoder_tokens = num_encoder_tokens
        self.num_decoder_tokens = num_decoder_tokens
        self.time_steps_encoder = time_steps_encoder
        self.max_probability = -1

        # models
        self.tokenizer, self.inf_encoder_model,  self.inf_decoder_model = inference_model()
        self.save_model_path = save_model_path
        self.test_path = test_path
        self.test_features_path = test_features_path
        self.search_type = 'greedy'

    def greedy_search(self, loaded_array):
        """

        :param f: the loaded numpy array after creating videos to frames and extracting features
        :return: the final sentence which has been predicted greedily
        """
        inv_map = self.index_to_word()
        states_value = self.inf_encoder_model.predict(loaded_array.reshape(-1, self.time_steps_encoder, self.num_encoder_tokens))
        target_seq = np.zeros((1, 1, self.num_decoder_tokens))
        final_sentence = ''
        target_seq[0, 0, self.tokenizer.word_index['bos']] = 1
        for i in range(15):
            output_tokens, h, c = self.inf_decoder_model.predict([target_seq] + states_value)
            states_value = [h, c]
            output_tokens = output_tokens.reshape(self.num_decoder_tokens)
            y_hat = np.argmax(output_tokens)
            if y_hat == 0:
                continue
            if inv_map[y_hat] is None:
                break
            if inv_map[y_hat] == 'eos':
                break
            else:
                final_sentence = final_sentence + inv_map[y_hat] + ' '
                target_seq = np.zeros((1, 1, self.num_decoder_tokens))
                target_seq[0, 0, y_hat] = 1
        return final_sentence

    def decode_sequence2bs(self, input_seq):
        states_value = self.inf_encoder_model.predict(input_seq)
        target_seq = np.zeros((1, 1, self.num_decoder_tokens))
        target_seq[0, 0, self.tokenizer.word_index['bos']] = 1
        self.beam_search(target_seq, states_value, [], [], 0)
        return decode_seq

    def beam_search(self, target_seq, states_value, prob, path, lens):
        """

        :param target_seq: the array that is fed into the model to predict the next word
        :param states_value: previous state that is fed into the lstm cell
        :param prob: probability of predicting a word
        :param path: list of words from each sentence
        :param lens: number of words
        :return: final sentence
        """
        global decode_seq
        node = 2
        output_tokens, h, c = self.inf_decoder_model.predict(
            [target_seq] + states_value)
        output_tokens = output_tokens.reshape(self.num_decoder_tokens)
        sampled_token_index = output_tokens.argsort()[-node:][::-1]
        states_value = [h, c]
        for i in range(node):
            if sampled_token_index[i] == 0:
                sampled_char = ''
            else:
                sampled_char = list(self.tokenizer.word_index.keys())[
                    list(self.tokenizer.word_index.values()).index(sampled_token_index[i])]
            MAX_LEN = 12
            if sampled_char != 'eos' and lens <= MAX_LEN:
                p = output_tokens[sampled_token_index[i]]
                if sampled_char == '':
                    p = 1
                prob_new = list(prob)
                prob_new.append(p)
                path_new = list(path)
                path_new.append(sampled_char)
                target_seq = np.zeros((1, 1, self.num_decoder_tokens))
                target_seq[0, 0, sampled_token_index[i]] = 1.
                self.beam_search(target_seq, states_value, prob_new, path_new, lens + 1)
            else:
                p = output_tokens[sampled_token_index[i]]
                prob_new = list(prob)
                prob_new.append(p)
                p = functools.reduce(operator.mul, prob_new, 1)
                if p > self.max_probability:
                    decode_seq = path
                    self.max_probability = p

    def decoded_sentence_tuning(self, decoded_sentence):
        # tuning sentence
        decode_str = []
        filter_string = ['bos', 'eos']
        uni_gram = {}
        last_string = ""
        for idx2, c in enumerate(decoded_sentence):
            if c in uni_gram:
                uni_gram[c] += 1
            else:
                uni_gram[c] = 1
            if last_string == c and idx2 > 0:
                continue
            if c in filter_string:
                continue
            if len(c) > 0:
                decode_str.append(c)
            if idx2 > 0:
                last_string = c
        return decode_str

    def index_to_word(self):
        # inverts word tokenizer
        index_to_word = {value: key for key, value in self.tokenizer.word_index.items()}
        return index_to_word

    def get_test_data(self, video_id):
        filename_test = []
        feat_test = []

        file_name = video_id.split(".")[0]
        filename_test.append(file_name)

        # getting cnn and yolo features for the test video
        cnn_path = os.path.join(self.test_features_path, 'feat', file_name + '.npy')
        yolo_path = os.path.join(self.test_features_path, 'yolo-feat', file_name + '.npy')
        if os.path.exists(cnn_path) and os.path.exists(yolo_path): #features exist
            cnn_f = np.load(cnn_path)
            yolo_f = np.load(yolo_path)
        else:
            print("features not found")
        yolo_f = yolo_f.reshape(-1, 30)
        #concatenate arrays
        combined_f = np.concatenate((cnn_f, yolo_f), axis=1)

        feat_test.append(combined_f)
        feat_test = np.array(feat_test)

        return feat_test, filename_test

    def test(self, video_id):
        feat_test, filename_test = self.get_test_data(video_id)

        for idx, feat in enumerate(feat_test):
                print(idx)
                print(f'Video: {filename_test[idx]}')
                if self.search_type == 'greedy':
                    start = time.time()
                    decoded_sentence = self.greedy_search(feat.reshape(-1, self.time_steps_encoder, self.num_encoder_tokens))
                    time_taken = '{:.2f}'.format(time.time()-start)
                    print(f'Caption: {decoded_sentence}')
                    print(f'Time taken: {time_taken}')

                # re-init max prob
                self.max_probability = -1

##Interactive

Model

In [110]:
chosen_model = interactive(f, x=models_list);
display(chosen_model)

interactive(children=(Dropdown(description='x', options=('without_cust_vocab_att_3700', 'without_cust_vocab_15…

In [117]:
chosen_model.result

'without_cust_vocab_3700'

In [118]:
save_model_path = f'/content/drive/Shareddrives/FYP-models/{chosen_model.result}'
encoder_name = 'encoder_model.h5'
decoder_name = 'decoder_model_weights.h5'
num_encoder_tokens = 4126
num_decoder_tokens = 3700
latent_dim = 512
time_steps_encoder = 80

In [119]:
model_name = save_model_path.split('/')[-1]
print(model_name)

without_cust_vocab_3700


Video

In [114]:
chosen_video = interactive(f, x=video_ids);
display(chosen_video)

interactive(children=(Dropdown(description='x', options=('fJr2evLANsE_0_10.avi', 'gqxpGOHUH9k_113_119.avi', 'f…

In [120]:
video_id = chosen_video.result

Generate Caption

In [121]:
if __name__ == "__main__":
    video_to_text = VideoDescriptionInference()
    video_to_text.test(video_id)

0
Video: fr9H1WLcF1A_141_148
1/1 [==============================] - 0s 30ms/step
Caption: a person is talking 
Time taken: 1.12
